In [1]:
# attack the SPEECHCOMMAND models

In [2]:
%load_ext autoreload
%autoreload 2
# Disable jedi autocompleter
%config Completer.use_jedi = False

In [3]:
import matplotlib as mpl
# set this 'backend' when using jupyter; do this before importing pyplot
mpl.use('nbagg')
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = (8.0, 6.0)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
# check gpus
!gpustat

crescent                Wed Nov 17 18:26:40 2021  418.152.00
[0] GeForce RTX 2080 Ti | 43'C,   0 % |   940 / 10989 MB | vietanh(929M)
[1] GeForce RTX 2080 Ti | 43'C,   0 % |    11 / 10989 MB |


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# manually set cuda device
# torch.cuda.set_device(1)
# device = 'cpu'
print(device)

cuda


In [7]:
import sys
sys.path.append('/home/felix/Research/Adversarial Research/FGN---Research/')
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh

In [9]:
sys.version_info  

sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)

In [8]:
# load dataset
batch_size = 32
batchsize_for_val = 128
(train_loader, val_loader, test_loader) = fgnh.SpeechCommands_Dataloaders(resample_rate = 8000,
                                                                          batch_size = batch_size,
                                                                          batchsize_for_val = batchsize_for_val,
                                                                          num_workers=5, 
                                                                          pin_memory=True)

In [9]:
# define model classes

## classic model
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2).squeeze()
    
# FGN model    
class FGN_M5(nn.Module):
    
    # changes:
    # nn.Conv1d -> fgnl.FGN_Conv1d
    # added g to conv inputs and outputs
    # make sure you pass g through the same pooling steps as x
    
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.fgn_conv1 = fgnl.FGN_Conv1d(in_channels=n_input, out_channels=n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.fgn_conv2 = fgnl.FGN_Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.fgn_conv3 = fgnl.FGN_Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.fgn_conv4 = fgnl.FGN_Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)
        
    def forward(self, x):
        x, g = self.fgn_conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        g = self.pool1(g)
        x, g = self.fgn_conv2(x, g)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        g = self.pool2(g)
        x, g = self.fgn_conv3(x ,g)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        g = self.pool3(g)
        x, _ = self.fgn_conv4(x, g)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2).squeeze()

In [10]:
# pretrained models paths
save_path = '../Experiments/sample_SPEECHCOMMANDS_models/'

classic_model_name= 'sample_classic_model_SPEECHCOMMANDS'
fgn_model_name = 'sample_FGN_model_SPEECHCOMMANDS'

In [11]:
# define and load the models
# classic model
classic_model = M5()
classic_model.load_state_dict(torch.load(save_path+classic_model_name+'_state_dict.pth'))
classic_model.to(device)

# fgn model trained from scratch
fgn_model_from_scratch = FGN_M5()
fgn_model_from_scratch.load_state_dict(torch.load(save_path+fgn_model_name+'_state_dict.pth'))
fgn_model_from_scratch.to(device)

# converted fgn model (no retraining)
fgn_model_converted_no_retraining = FGN_M5()
fgn_model_converted_no_retraining.load_state_dict(torch.load(save_path+'sample_FGN_converted_model_SPEECHCOMMANDS'+'_state_dict.pth'))
fgn_model_converted_no_retraining.to(device)

# converted and retrained 1 epoch fgn model
fgn_model_converted_fast_retraining = FGN_M5()
fgn_model_converted_fast_retraining.load_state_dict(torch.load(save_path+'sample_FGN_converted_fast_retrained_model_SPEECHCOMMANDS'+'_state_dict.pth'))
fgn_model_converted_fast_retraining.to(device)

# converted and retrained 21 epoch fgn model
fgn_model_converted_long_retraining = FGN_M5()
fgn_model_converted_long_retraining.load_state_dict(torch.load(save_path+'sample_FGN_converted_long_retrained_model_SPEECHCOMMANDS'+'_state_dict.pth'))
fgn_model_converted_long_retraining.to(device)

/home/felix/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:125: UserWarning: 
GeForce RTX 2080 Ti with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the GeForce RTX 2080 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


FGN_M5(
  (fgn_conv1): FGN_Conv1d(
    (Conv1d): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  )
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv2): FGN_Conv1d(
    (Conv1d): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  )
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv3): FGN_Conv1d(
    (Conv1d): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  )
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv4): FGN_Conv1d(
    (Conv1d): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  )
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): Ma

In [12]:
# set all models to eval mode
classic_model.eval()
fgn_model_from_scratch.eval()
fgn_model_converted_no_retraining.eval()
fgn_model_converted_fast_retraining.eval()
fgn_model_converted_long_retraining.eval()

FGN_M5(
  (fgn_conv1): FGN_Conv1d(
    (Conv1d): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  )
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv2): FGN_Conv1d(
    (Conv1d): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  )
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv3): FGN_Conv1d(
    (Conv1d): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  )
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv4): FGN_Conv1d(
    (Conv1d): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  )
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): Ma

In [13]:
# functions to test models
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)


def test(model, loader):
    model.eval()
    correct = 0
    for data, target in loader:

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        output = model(data)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

    print(f'Accuracy: {correct}/{len(loader.dataset)} ({100. * correct / len(loader.dataset):.0f}%)')

In [14]:
# # # verify accuracies
# print('Train/Test/Val accuracy for Classic Model')
# test(classic_model, train_loader)
# test(classic_model, val_loader)
# test(classic_model, test_loader)

# print('Train/Test/Val accuracy for FGN model trained from scratch')
# test(fgn_model_from_scratch, train_loader)
# test(fgn_model_from_scratch, val_loader)
# test(fgn_model_from_scratch, test_loader)

# print('Train/Test/Val accuracy for FGN model converted from classic (no retraining)')
# test(fgn_model_converted_no_retraining, train_loader)
# test(fgn_model_converted_no_retraining, val_loader)
# test(fgn_model_converted_no_retraining, test_loader)

# print('Train/Test/Val accuracy for FGN model converted from classic (fast retraining)')
# test(fgn_model_converted_fast_retraining, train_loader)
# test(fgn_model_converted_fast_retraining, val_loader)
# test(fgn_model_converted_fast_retraining, test_loader)

# print('Train/Test/Val accuracy for FGN model converted from classic (long retraining)')
# test(fgn_model_converted_long_retraining, train_loader)
# test(fgn_model_converted_long_retraining, val_loader)
# test(fgn_model_converted_long_retraining, test_loader)

In [15]:
# ### Results (no need to rerun):
# Train/Test/Val accuracy for Classic Model
# Accuracy: 76655/84843 (90%)
# Accuracy: 8755/9981 (88%)
# Accuracy: 9468/11005 (86%)
# Train/Test/Val accuracy for FGN model trained from scratch
# Accuracy: 73489/84843 (87%)
# Accuracy: 8572/9981 (86%)
# Accuracy: 9244/11005 (84%)
# Train/Test/Val accuracy for FGN model converted from classic (no retraining)
# Accuracy: 76655/84843 (90%)
# Accuracy: 8755/9981 (88%)
# Accuracy: 9468/11005 (86%)
# Train/Test/Val accuracy for FGN model converted from classic (fast retraining)
# Accuracy: 76264/84843 (90%)
# Accuracy: 8654/9981 (87%)
# Accuracy: 9394/11005 (85%)
# Train/Test/Val accuracy for FGN model converted from classic (long retraining)
# Accuracy: 77561/84843 (91%)
# Accuracy: 8726/9981 (87%)
# Accuracy: 9411/11005 (86%)

In [16]:
### Start Attacking the models

In [17]:
import foolbox

In [18]:
import numpy as np

In [19]:
# set model bounds and preprocessing

# precomputed bounds min and max input values
min_bound = -1.3844940662384033
max_bound = 1.3773366212844849

bounds = (min_bound, max_bound)
# preprocessing - I think these would be used in similar way to pytorch preprocessing
# but possible passed to whatever architecture is used (torch, tensorflow, other) 
# in my case the dataloaders already normalizes the data
preprocessing = dict(mean=0, std=1)

In [20]:
# ready the models for foolbox
classic_f_model = foolbox.PyTorchModel(classic_model, bounds=bounds,
                                       preprocessing=preprocessing, device=device)

fgn_f_model_from_scratch = foolbox.PyTorchModel(fgn_model_from_scratch, bounds=bounds,
                                       preprocessing=preprocessing, device=device)

fgn_f_model_converted_no_retraining = foolbox.PyTorchModel(fgn_model_converted_no_retraining, bounds=bounds,
                                       preprocessing=preprocessing, device=device)

fgn_f_model_converted_fast_retraining = foolbox.PyTorchModel(fgn_model_converted_fast_retraining, bounds=bounds,
                                       preprocessing=preprocessing, device=device)

fgn_f_model_converted_long_retraining = foolbox.PyTorchModel(fgn_model_converted_long_retraining, bounds=bounds,
                                       preprocessing=preprocessing, device=device)


In [21]:
from tqdm.notebook import tqdm

In [22]:
# debug tool
import itertools
# 
start = 0
stop = 36 # can be None

In [23]:
# foolbox.accuracy doesn't work with dataloaders, so building a custom func to do so
def f_accuracy(model, dataloader, proc_func=None):
    # given a model and a dataloader, computes accuracy
    # proc_func is a processing function to apply to the values of the dataloader
    # that returns (inputs, targets)
    
    # get model device
    device = model.device
    
    running_count = 0
    running_average = 0
    # go through the dataset (assumes inputs and target are what is returned )
    for batch in tqdm(itertools.islice(dataloader, start=start, end=end)):
        # apply proc_func 
        if proc_func != None:
            inputs, targets = proc_func(*batch)
        else:
            inputs, targets = batch
        
        # send data to proper device
        inputs = inputs.to(device)
        targets = targets.to(device)

        # update running average accuracy and count
        running_average = (len(inputs)*foolbox.utils.accuracy(model, inputs, targets) + running_count*running_average)/(len(inputs)+running_count)
        running_count += len(inputs)
    
    return(running_average)


In [24]:
# # check model accuracies
# print('Train/Test/Val accuracy for Classic Model')
# print(f_accuracy(classic_f_model, train_loader))
# print(f_accuracy(classic_f_model, val_loader))
# print(f_accuracy(classic_f_model, test_loader))
# print('Train/Test/Val accuracy for FGN model trained from scratch')
# print(f_accuracy(fgn_f_model_from_scratch, train_loader))
# print(f_accuracy(fgn_f_model_from_scratch, val_loader))
# print(f_accuracy(fgn_f_model_from_scratch, test_loader))
# print('Train/Test/Val accuracy for FGN model converted from classic (no retraining)')
# print(f_accuracy(fgn_f_model_converted_no_retraining, train_loader))
# print(f_accuracy(fgn_f_model_converted_no_retraining, val_loader))
# print(f_accuracy(fgn_f_model_converted_no_retraining, test_loader))
# print('Train/Test/Val accuracy for FGN model converted from classic (fast retraining)')
# print(f_accuracy(fgn_f_model_converted_fast_retraining, train_loader))
# print(f_accuracy(fgn_f_model_converted_fast_retraining, val_loader))
# print(f_accuracy(fgn_f_model_converted_fast_retraining, test_loader))
# print('Train/Test/Val accuracy for FGN model converted from classic (long retraining)')
# print(f_accuracy(fgn_f_model_converted_long_retraining, train_loader))
# print(f_accuracy(fgn_f_model_converted_long_retraining, val_loader))
# print(f_accuracy(fgn_f_model_converted_long_retraining, test_loader))

In [25]:
# Results (should be close to identical)

In [26]:
### looks like they are the same, continue with attacks

In [27]:
# attack params to explore
epsilons = torch.tensor([(max_bound-min_bound)*x 
            for x in 
            [0.0,
             1/256,
             3/512,
             1/128,
             3/256,
             1/64,
             3/128,
             1/32,
             3/64,
             1/16,
             3/32,
             1/8,
             3/16,
             1/4,
             3/8,
             1/2,
             3/4,
             1.0,] ], device=device)

print('epsilons: {}'.format(epsilons))

epsilons: tensor([0.0000, 0.0108, 0.0162, 0.0216, 0.0324, 0.0432, 0.0647, 0.0863, 0.1295,
        0.1726, 0.2589, 0.3452, 0.5178, 0.6905, 1.0357, 1.3809, 2.0714, 2.7618],
       device='cuda:0')


In [28]:
# write a function that attacks a model using a dataloader

def perform_attack(attack_func, f_model, dataloader):
    # iterates over dataloader 
    # attack is the attack function AFTER being defined: ex LinfPGD_attack=foolbox.attacks.LinfPGD()
    # (so it's the output foolbox.attacks.LinfPGD(), not foolbox.attacks.LinfPGD itself)
    # ensure the dataloader iterator returns (inputs, labels) 
    
    # defines results to return, shape is (epsilons, sample, (sample shape))=(18,32xbatches,1,8000)
    num_epsilons = 18 # hardcoded for now
    data_shape = (1,8000) # next(iter(dataloader))[0].shape[1:] # this could be expensive, hardcoded for now
    # create empty lists of the right shape
    results = {'adv_raw':np.array([]).reshape((num_epsilons, 0, *(data_shape))),
               'adv_clipped':np.array([]).reshape((num_epsilons, 0, *(data_shape))),
               'adv_success':np.array([]).reshape((num_epsilons, 0))}
    
    # iterate over loader
    for inputs, labels in tqdm(itertools.islice(dataloader, start, stop)):
        
        # attack
        adv_raw, adv_clipped, adv_success = attack_func(f_model = f_model, 
                                                        inputs = inputs, 
                                                        labels =labels
                                                       )
        # compile with results
        results['adv_raw'] = np.concatenate([results['adv_raw'],
                                             np.array([x.cpu().numpy() for x in adv_raw])],
                                            axis=1)
        results['adv_clipped'] = np.concatenate([results['adv_clipped'],
                                                 np.array([x.cpu().numpy() for x in adv_clipped])],
                                                axis=1)
        results['adv_success'] = np.concatenate([results['adv_success'],
                                                 np.array([x.cpu().numpy() for x in adv_success])],
                                                axis=1)
    
    # return results dictionary
    return(results)

In [29]:
### Now, perform the attacks on the models, saving the results 

In [30]:
### attack parameters
L2CarliniWagner_attack=foolbox.attacks.L2CarliniWagnerAttack()
LinfPGD_attack=foolbox.attacks.LinfPGD()


# targetted vs untargetted
from foolbox.criteria import Misclassification

# define the entire attack function using epsilons, criterion,
def L2CarliniWagner_attack_func(f_model, inputs, labels):
    device = f_model.device
    inputs = inputs.to(device)
    criterions = Misclassification(labels.to(device))
    return L2CarliniWagner_attack(model=f_model, inputs=inputs, criterion=criterions, epsilons=epsilons)

def LinfPGD_attack_func(f_model, inputs, labels):
    device = f_model.device
    inputs = inputs.to(device)
    criterions = Misclassification(labels.to(device))
    return LinfPGD_attack(model=f_model, inputs=inputs, criterion=criterions, epsilons=epsilons)

In [31]:
# name for the models we are attacking
models_to_attack = {'classic_f_model':classic_f_model, 
                    'fgn_f_model_from_scratch':fgn_f_model_from_scratch, 
                    'fgn_f_model_converted_no_retraining':fgn_f_model_converted_no_retraining,
                    'fgn_f_model_converted_fast_retraining':fgn_f_model_converted_fast_retraining,
                    'fgn_f_model_converted_long_retraining':fgn_f_model_converted_long_retraining
                   }
# names of funcs for attacks
attacks_to_perform = {'L2CarliniWagner':L2CarliniWagner_attack_func,
                     'LinfPGD':LinfPGD_attack_func}

In [32]:
from time import time
import os
import pickle
import gc

In [33]:
# timestamp
timestamp = time()
save_folder = '../Experiments/adversarial_attacks_results/{}/'.format(timestamp)
os.makedirs(save_folder)

In [ ]:
# for attack_name, attack in attacks_to_perform.items():
#     print('Performing attack:', attack_name)
#     for model_name, f_model in models_to_attack.items():
#         if not os.path.exists(save_folder+'{}_{}_{}.pickle'.format(attack_name, model_name, 'adv_raw')):
#             print('Attacking', model_name)

#             # do attack
#             results = perform_attack(attack, f_model, val_loader)

#             # save results
#             # save files separately (can be as big as 11GB)
#             for adv_name in ['adv_raw', 'adv_clipped', 'adv_success']:
#                 with open(save_folder+'{}_{}_{}.pickle'.format(attack_name, model_name, adv_name), 'wb') as f:
#                     pickle.dump(results[adv_name], f, protocol=4)

#             # delete objects (might help the Garbage Collector free up space)
# # #             del(results)
# # #             del(f_model)
# # #             torch.cuda.empty_cache()
# # #             del(val_loader)
# # #             gc.collect()
# # #             (_, val_loader, _) = fgnh.SpeechCommands_Dataloaders(resample_rate = 8000,
# #                                                                           batch_size = batch_size,
# #                                                                           batchsize_for_val = batchsize_for_val,
# #                                                                           num_workers=5, 
# #                                                                           pin_memory=True)
            
#         else:
#             print('skipping')
#             print(save_folder+'{}_{}_{}.pickle'.format(attack_name, model_name, 'adv_raw'))

Performing attack: L2CarliniWagner
Attacking classic_f_model


In [1]:
!gpustat

crescent                Mon Feb 21 15:58:41 2022  418.152.00
[0] GeForce RTX 2080 Ti | 31'C,   0 % |     0 / 10989 MB |
[1] GeForce RTX 2080 Ti | 29'C,   0 % |     0 / 10989 MB |


In [2]:
!nvidia-smi

Mon Feb 21 15:58:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.152.00   Driver Version: 418.152.00   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:04:00.0 Off |                  N/A |
|  0%   31C    P8    21W / 260W |      0MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:83:00.0 Off |                  N/A |
|  0%   29C    P8     1W / 260W |      0MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        